In [ ]:
import numpy as np
import librosa
import os
from matplotlib import pyplot as plt
!pip install pretty_midi
import pretty_midi 
pretty_midi.pretty_midi.MAX_TICK = 1e10

     |████████████████████████████████| 5.6 MB 4.6 MB/s 
     |████████████████████████████████| 51 kB 6.9 MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591955 sha256=ccc1eba18aa90a41bc0bcf4db2c2660fe616f616eb0a0644fc07b03a4e8e9684
  Stored in directory: /root/.cache/pip/wheels/ad/74/7c/a06473ca8dcb63efb98c1e67667ce39d52100f837835ea18fa
Successfully built pretty-midi


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#preparing the data from the original audio files
entries = os.scandir('/content/drive/MyDrive/train_source')#loading the data to process which we are going to use for training 

for file in entries:
    file_name,file_extensions=os.path.splitext(file)

    if file_extensions=='.wav':
        sr=16000
        y, sr = librosa.load(file, sr=sr)
        cqt_file=librosa.core.cqt(x, sr=16000,fmin=None, n_bins=252, bins_per_octave=36)#using the librosa library to calculate the cqt of the audio file which is returned as a 2D vector 
        cqt=np.abs(cqt_file)# the returned 2d vector is a complex quantity 
        cqt=librosa.amplitude_to_db(cqt)# taking the amplitude on logarithmic scale gives a better visualization of data
        b=np.shape(cqt)[1]
        print(np.shape(np.transpose(cqt)))
        entries1 = os.scandir('/content/drive/MyDrive/train_source')#it is being loaded again so that we can get the corresponding midi file to the wav file which we have just used
        for file1 in entries1:
          file_name1,file_extensions1=os.path.splitext(file1)

          if file_name1==file_name and file_extensions1==".mid":
            file2=file_name + file_extensions1
            pm = pretty_midi.PrettyMIDI(file2)
            a=pm.instruments[0].get_piano_roll(float(sr*float(b)/float(c)))# The Sampling frequency has been taken such that its shape and the cqt files shape is same 
            a = (a[21:109, :b] > 0)#we only need the frequency of the piano and converting non binary data to 0 and 1
            a=np.transpose(a)
            print(np.shape(a))
            out = '/content/drive/MyDrive/trainfiles/trainmid/mid_'+str(file_name1[40:])+'mid'+'.npy'
            np.save(out,a)
            out = '/content/drive/MyDrive/trainfiles/trainwav/wav_'+str(file_name[40:])+'.npy'
            cqt=np.transpose(cqt)
            if a.shape[0]<cqt.shape[0]: #cutting out little extra part in CQT array
              cqt=cqt[:a.shape[0],:]
            print(np.shape(cqt))  
            np.save(out,cqt)
            print('wrote file', out)
            print("saving", out)

  

(7468, 252)
(7405, 88)
(7405, 252)
wrote file /content/drive/MyDrive/trainfiles/trainwav/wav__MUS-alb_se2_StbgTGd2.npy
saving /content/drive/MyDrive/trainfiles/trainwav/wav__MUS-alb_se2_StbgTGd2.npy
(12058, 252)
(11995, 88)
(11995, 252)
wrote file /content/drive/MyDrive/trainfiles/trainwav/wav__MUS-bk_xmas1_StbgTGd2.npy
saving /content/drive/MyDrive/trainfiles/trainwav/wav__MUS-bk_xmas1_StbgTGd2.npy
(7215, 252)
(7152, 88)
(7152, 252)
wrote file /content/drive/MyDrive/trainfiles/trainwav/wav__MUS-bk_xmas4_StbgTGd2.npy
saving /content/drive/MyDrive/trainfiles/trainwav/wav__MUS-bk_xmas4_StbgTGd2.npy
(12370, 252)
(12307, 88)
(12307, 252)
wrote file /content/drive/MyDrive/trainfiles/trainwav/wav__MUS-bk_xmas5_StbgTGd2.npy
saving /content/drive/MyDrive/trainfiles/trainwav/wav__MUS-bk_xmas5_StbgTGd2.npy
(3595, 252)
(3532, 88)
(3532, 252)
wrote file /content/drive/MyDrive/trainfiles/trainwav/wav__MUS-bor_ps6_StbgTGd2.npy
saving /content/drive/MyDrive/trainfiles/trainwav/wav__MUS-bor_ps6_StbgTG

In [ ]:
path1='/content/drive/MyDrive/trainfiles/trainwav'

def concatenate_wav(path):#we are dividing the number of features of the cqt file into batches of 100 and stacking them up to create a 3D array 
  entries = os.scandir(path)
  d=np.zeros((1,100,252))#initialising the array with the correct shape 

  for file in entries:
      a=np.load(file)
      a=a[np.newaxis,:,:]# adding a new axis to the original files
      for b in range(int(np.shape(a)[1]/100)):
          
          c=a[:,b*100:(b*100)+100,:]# dividing them into batches of 100
          d=np.concatenate((d,c), axis=0)# stacking them up on the first axis 
      print(np.shape(d)) 
  out = '/content/drive/MyDrive/trainfiles/trainwav/concatenate'+'wav'+'.npy'
  d=d[1:,:,:]# array was initialise with zeros so excluding that part 

  np.save(out,d)  
  print(np.shape(d))

concatenate_wav(path1)

path='/content/drive/MyDrive/trainfiles/trainmid'

def concatenate_mid(path):#we are dividing the number of features of the pianoroll file into batches of 100 and stacking them up to create a 3D array 
  entries = os.scandir(path)
  d=np.zeros((1,100,88))
  l=0
  for file in entries:
      a=np.load(file)
      a=a[np.newaxis,:,:]
      for b in range(int(np.shape(a)[1]/100)):
            
          c=a[:,b*100:(b*100)+100,:]
          d=np.concatenate((d,c), axis=0)
      print(np.shape(d))      

          
  out = '/content/drive/MyDrive/trainfiles/trainmid/concatenate'+'mid'+'.npy'
  d=d[1:,:,:]
  np.save(out,d)  

concatenate_mid(path)


(75, 100, 252)
(194, 100, 252)
(265, 100, 252)
(388, 100, 252)
(423, 100, 252)
(457, 100, 252)
(478, 100, 252)
(555, 100, 252)
(585, 100, 252)
(662, 100, 252)
(763, 100, 252)
(960, 100, 252)
(1041, 100, 252)
(1155, 100, 252)
(1214, 100, 252)
(1297, 100, 252)
(1460, 100, 252)
(1568, 100, 252)
(1601, 100, 252)
(1749, 100, 252)
(1923, 100, 252)
(2025, 100, 252)
(2170, 100, 252)
(2194, 100, 252)
(2220, 100, 252)
(2299, 100, 252)
(2363, 100, 252)
(2405, 100, 252)
(2465, 100, 252)
(2542, 100, 252)
(2612, 100, 252)
(2671, 100, 252)
(2794, 100, 252)
(2954, 100, 252)
(2971, 100, 252)
(3018, 100, 252)
(3026, 100, 252)
(3035, 100, 252)
(3073, 100, 252)
(3089, 100, 252)
(3120, 100, 252)
(3147, 100, 252)
(3174, 100, 252)
(3262, 100, 252)
(3307, 100, 252)
(3340, 100, 252)
(3387, 100, 252)
(3510, 100, 252)
(3541, 100, 252)
(3655, 100, 252)
(3888, 100, 252)
(3997, 100, 252)
(4094, 100, 252)
(4103, 100, 252)
(4143, 100, 252)
(4280, 100, 252)
(4315, 100, 252)
(4375, 100, 252)
(4452, 100, 252)
(4519, 100

In [ ]:
#preparing the data from the original audio files
entries = os.scandir('/content/drive/MyDrive/test_source')#loading the data to process which we are going to use for training 

for file in entries:
    file_name,file_extensions=os.path.splitext(file)

    if file_extensions=='.wav':
        sr=16000
        y, sr = librosa.load(file, sr=sr)
        cqt_file=librosa.cqt(y, sr=16000,fmin=None, n_bins=252, bins_per_octave=36)#using the librosa library to calculate the cqt of the audio file which is returned as a 2D vector 
        c=len(y) # it is being stored because it will be used for the frequency while making the piano roll file
        cqt=np.abs(cqt_file)# the returned 2d vector is a complex quantity 
        cqt=librosa.amplitude_to_db(cqt)# taking the amplitude on logarithmic scale gives a better visualization of data
        b=np.shape(cqt)[1]
        print(np.shape(np.transpose(cqt)))
        entries1 = os.scandir('/content/drive/MyDrive/test_source')#it is being loaded again so that we can get the corresponding midi file to the wav file which we have just used
        for file1 in entries1:
          file_name1,file_extensions1=os.path.splitext(file1)

          if file_name1==file_name and file_extensions1==".mid":
            file2=file_name + file_extensions1
            pm = pretty_midi.PrettyMIDI(file2)
            a=pm.instruments[0].get_piano_roll(float(sr*float(b)/float(c)))# The Sampling frequency has been taken such that its shape and the cqt files shape is same 
            a = (a[21:109, :b] > 0)#we only need the frequency of the piano and converting non binary data to 0 and 1
            a=np.transpose(a)
            print(np.shape(a))
            out = '/content/drive/MyDrive/testfiles/testmid/mid_'+str(file_name1[40:])+'mid'+'.npy'
            np.save(out,a)
            out = '/content/drive/MyDrive/testfiles/testwav/wav_'+str(file_name[40:])+'.npy'
            cqt=np.transpose(cqt)
            if a.shape[0]<cqt.shape[0]: #cutting out little extra part in CQT array
              cqt=cqt[:a.shape[0],:]
            print(np.shape(cqt))  
            np.save(out,cqt)
            print('wrote file', out)
            print("saving", out)

  

(7458, 252)
(7411, 88)
(7411, 252)
wrote file /content/drive/MyDrive/testfiles/testwav/wav_MUS-alb_se2_ENSTDkCl.npy
saving /content/drive/MyDrive/testfiles/testwav/wav_MUS-alb_se2_ENSTDkCl.npy
(12048, 252)
(12001, 88)
(12001, 252)
wrote file /content/drive/MyDrive/testfiles/testwav/wav_MUS-bk_xmas1_ENSTDkCl.npy
saving /content/drive/MyDrive/testfiles/testwav/wav_MUS-bk_xmas1_ENSTDkCl.npy
(7204, 252)
(7157, 88)
(7157, 252)
wrote file /content/drive/MyDrive/testfiles/testwav/wav_MUS-bk_xmas4_ENSTDkCl.npy
saving /content/drive/MyDrive/testfiles/testwav/wav_MUS-bk_xmas4_ENSTDkCl.npy
(12360, 252)
(12313, 88)
(12313, 252)
wrote file /content/drive/MyDrive/testfiles/testwav/wav_MUS-bk_xmas5_ENSTDkCl.npy
saving /content/drive/MyDrive/testfiles/testwav/wav_MUS-bk_xmas5_ENSTDkCl.npy
(3585, 252)
(3538, 88)
(3538, 252)
wrote file /content/drive/MyDrive/testfiles/testwav/wav_MUS-bor_ps6_ENSTDkCl.npy
saving /content/drive/MyDrive/testfiles/testwav/wav_MUS-bor_ps6_ENSTDkCl.npy
(3521, 252)
(3474, 88)


In [ ]:
path1='/content/drive/MyDrive/testfiles/testwav'

def concatenate_wav(path):#we are dividing the number of features of the cqt file into batches of 100 and stacking them up to create a 3D array 
  entries = os.scandir(path)
  d=np.zeros((1,100,252))#initialising the array with the correct shape 

  for file in entries:
      a=np.load(file)
      a=a[np.newaxis,:,:]# adding a new axis to the original files
      for b in range(int(np.shape(a)[1]/100)):
          
          c=a[:,b*100:(b*100)+100,:]# dividing them into batches of 100
          d=np.concatenate((d,c), axis=0)# stacking them up on the first axis 
  out = '/content/drive/MyDrive/testfiles/testwav/concatenate'+'wav'+'.npy'
  d=d[1:,:,:]# array was initialise with zeros so excluding that part 

  np.save(out,d)  

concatenate_wav(path1)

path='/content/drive/MyDrive/testfiles/testmid'

def concatenate_mid(path):#we are dividing the number of features of the pianoroll file into batches of 100 and stacking them up to create a 3D array 
  entries = os.scandir(path)
  d=np.zeros((1,100,88))
  l=0
  for file in entries:
      a=np.load(file)
      a=a[np.newaxis,:,:]
      for b in range(int(np.shape(a)[1]/100)):
            
          c=a[:,b*100:(b*100)+100,:]
          d=np.concatenate((d,c), axis=0)     

          
  out = '/content/drive/MyDrive/testfiles/testmid/concatenate'+'mid'+'.npy'
  d=d[1:,:,:]
  np.save(out,d)  

concatenate_mid(path)
